## Load modules

In [13]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta, date, datetime

## Combine all weeks

In [2]:
base_name='pulse2020_puf_'
df_lists=[]

for f in os.listdir('../data'):
    if base_name in f:
        data=pd.read_csv(os.path.join('../data',f))
        #print(data.head())
        df_lists.append(data)
final_data = pd.concat(df_lists)
# final_data.to_excel(os.path.join('../data',base_name+'all.xlsx'))

## Data Cleaning

Check which any value is empty

In [3]:
final_data.isnull().values.any()

True

Check how many values are empty

In [4]:
 final_data.isnull().sum().sum()

123743289

Check which columns are empty

In [5]:
empty_cols = [col for col in final_data.columns if final_data[col].isnull().any()]
print(empty_cols)

['EST_MSA', 'MORTLMTH', 'TSCHLHRS', 'TTCH_HRS', 'CHILDFOOD', 'TSTDY_HRS', 'EIP', 'EIPSPND1', 'EIPSPND2', 'EIPSPND3', 'EIPSPND4', 'EIPSPND5', 'EIPSPND6', 'EIPSPND7', 'EIPSPND8', 'EIPSPND9', 'EIPSPND10', 'EIPSPND11', 'EIPSPND12', 'EIPSPND13', 'SPNDSRC1', 'SPNDSRC2', 'SPNDSRC3', 'SPNDSRC4', 'SPNDSRC5', 'SPNDSRC6', 'SPNDSRC7', 'REGION', 'HWEIGHT', 'TW_START', 'UI_APPLY', 'UI_RECV', 'TUI_NUMPER', 'SSA_RECV', 'SSA_APPLY', 'SSAPGM1', 'SSAPGM2', 'SSAPGM3', 'SSAPGM4', 'SSAPGM5', 'SSALIKELY', 'SSAEXPCT1', 'SSAEXPCT2', 'SSAEXPCT3', 'SSAEXPCT4', 'SSAEXPCT5', 'SSADECISN', 'EXPNS_DIF', 'CHNGHOW1', 'CHNGHOW2', 'CHNGHOW3', 'CHNGHOW4', 'CHNGHOW5', 'CHNGHOW6', 'CHNGHOW7', 'CHNGHOW8', 'CHNGHOW9', 'CHNGHOW10', 'CHNGHOW11', 'CHNGHOW12', 'WHYCHNGD1', 'WHYCHNGD2', 'WHYCHNGD3', 'WHYCHNGD4', 'WHYCHNGD5', 'WHYCHNGD6', 'WHYCHNGD7', 'WHYCHNGD8', 'WHYCHNGD9', 'WHYCHNGD10', 'WHYCHNGD11', 'WHYCHNGD12', 'WHYCHNGD13', 'SPNDSRC8', 'FEWRTRIPS', 'FEWRTRANS', 'PLNDTRIPS', 'CNCLDTRPS', 'SNAP_YN', 'SNAPMNTH1', 'SNAPMNTH2', 

For each columns, how many values are missing

In [6]:
for col in empty_cols:
    print(col,(final_data[col].isnull().sum())/(1.0*len(final_data)))


EST_MSA 0.6921089225090093
MORTLMTH 0.09107582065619088
TSCHLHRS 0.09107582065619088
TTCH_HRS 0.09107582065619088
CHILDFOOD 0.3805447795784911
TSTDY_HRS 0.3805447795784911
EIP 0.5411917001081542
EIPSPND1 0.5411917001081542
EIPSPND2 0.5411917001081542
EIPSPND3 0.5411917001081542
EIPSPND4 0.5411917001081542
EIPSPND5 0.5411917001081542
EIPSPND6 0.5411917001081542
EIPSPND7 0.5411917001081542
EIPSPND8 0.5411917001081542
EIPSPND9 0.5411917001081542
EIPSPND10 0.5411917001081542
EIPSPND11 0.5411917001081542
EIPSPND12 0.5411917001081542
EIPSPND13 0.5411917001081542
SPNDSRC1 0.4501158794519633
SPNDSRC2 0.4501158794519633
SPNDSRC3 0.4501158794519633
SPNDSRC4 0.4501158794519633
SPNDSRC5 0.4501158794519633
SPNDSRC6 0.4501158794519633
SPNDSRC7 0.4501158794519633
REGION 0.9089241793438091
HWEIGHT 0.9089241793438091
TW_START 0.9089241793438091
UI_APPLY 0.9089241793438091
UI_RECV 0.9089241793438091
TUI_NUMPER 0.9089241793438091
SSA_RECV 0.9089241793438091
SSA_APPLY 0.9089241793438091
SSAPGM1 0.90892417

As per the field, might need pre-processing

In [8]:
final_data = final_data[((final_data.ANXIOUS != -99) & (final_data.ANXIOUS!= -88)) &
               ((final_data.WORRY != -99) & (final_data.WORRY != -88)) &
               ((final_data.INTEREST != -99) & (final_data.INTEREST != -88)) &
               ((final_data.DOWN != -99) & (final_data.DOWN != -88))]

final_data.reset_index(inplace=True, drop=True)

In [42]:
final_data['Anxiety'] = final_data['ANXIOUS'] + final_data['WORRY'] - 2
final_data['Depression'] = final_data['INTEREST'] + final_data['DOWN'] - 2

In [ ]:
def anxiety_disorder(anxiety_score):
    """ The sum of the variables ANXIOUS AND WORRY - 2 must be at least 3 to consider an anxiety disorder
    """
    return True if anxiety_score >= 3 else False

def depression_disorder(depression_score):
    """ The sum of the variables INTEREST AND DOWN - 2 must be at least 3 to consider an anxiety disorder
    """
    return True if depression_score >= 3 else False

In [ ]:
final_data['anxiety_disorder'] = final_data.Anxiety.apply(anxiety_disorder)
final_data['depression_disorder'] = final_data.Depression.apply(anxiety_disorder)

Include the age

In [16]:
final_data['AGE']=datetime.now().year-final_data['TBIRTH_YEAR']

count    1.079133e+06
mean     5.164927e+01
std      1.559702e+01
min      1.800000e+01
25%      3.900000e+01
50%      5.200000e+01
75%      6.400000e+01
max      8.800000e+01
Name: AGE, dtype: float64

Manually adding mid dates of each period due to lack of continuity

In [40]:
import datetime

dates=[date(2020,4,29),date(2020,5,9),date(2020,5,14),date(2020,5,23),date(2020,5,30),date(2020,6,6),date(2020,6,13),\
       date(2020,6,20),date(2020,6,27),date(2020,7,4),date(2020,7,11),date(2020,7,18),date(2020,8,25),date(2020,9,8)]

final_data['DATE']=''
for i,dt in enumerate(dates):
    idx_dt=final_data.index[final_data['WEEK'] == i+1].tolist()
    final_data['DATE'][idx_dt]=dt


c:\users\ee11b\lung and glioma\venv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\ee11b\lung and glioma\venv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\users\ee11b\lung and glioma\venv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

c:\users\ee11b\lung and glioma\venv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\ee11b\lung and glioma\venv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\users\ee11b\lung and glioma\venv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [41]:
final_data.to_excel(os.path.join('../data',base_name+'all.xlsx'))

ValueError: This sheet is too large! Your sheet size is: 1079133, 207 Max sheet size is: 1048576, 16384